In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from heystox_intraday.trading_indicator import *

In [2]:
raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
sorted_stock = SortedStocksList.objects.get(symbol__symbol="spencers", created_at__date=datetime.now().date())

In [3]:
macd_indicator = Indicator.objects.get(name="MACD")
df = get_macd_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df[75:]
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
df_after_last_crossover = df.loc[df["date"] >= last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
except:
    crossover_signal = None
# df.head(50)
df.loc[(df.macd_diff >= 0.070)]
last_crossover
crossover_signal

candle_type                             M5
open_price                           94.15
high_price                           96.45
low_price                               94
close_price                          96.35
volume                              434513
total_buy_quantity                    None
total_sell_quantity                   None
date                   2020-01-27 13:40:00
macd                              0.457843
macd_signal                       0.250695
macd_diff                         0.207148
percentage                        0.000948
signal                                 BUY
Name: 128, dtype: object

In [4]:
macd_indicator = Indicator.objects.get(name="MACD")
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df[75:]
if sorted_stock.entry_type in ["SELL", "SS"]:
    last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
elif sorted_stock.entry_type in ["BUY", "SB"]:
    last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
df_after_last_crossover = df.loc[df["date"] >= last_crossover.date]
try:
    if last_crossover.signal == "SELL_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
    elif last_crossover.signal == "BUY_CROSSOVER":
        crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 22.70)].iloc[0]
except:
    crossover_signal = None
df_after_last_crossover

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
147,M5,94.15,94.80,94.0,94.75,27671,None,None,2020-01-27 15:15:00,25.000000,12.408451,12.591549,3.147887,BUY_CROSSOVER
148,M5,94.90,94.95,94.3,94.55,48368,None,None,2020-01-27 15:20:00,19.642857,16.289403,3.353454,0.658714,BUY
149,M5,94.55,95.00,94.4,94.80,92665,None,None,2020-01-27 15:25:00,28.571429,24.404762,4.166667,1.190476,BUY


In [5]:
raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [6]:
from heystox_intraday.intraday_functions_strategy import get_stochastic_crossover, get_macd_crossover

candle_type                             M5
open_price                           94.15
high_price                           96.45
low_price                               94
close_price                          96.35
volume                              434513
total_buy_quantity                    None
total_sell_quantity                   None
date                   2020-01-27 13:40:00
macd                              0.457843
macd_signal                       0.250695
macd_diff                         0.207148
percentage                        0.000948
signal                                 BUY
Name: 128, dtype: object